# Quick Sample Flow - All Models Demo

This section demonstrates a quick sample run of all implemented model classes with minimal data and iterations to verify everything works correctly.

In [1]:
# Imports

from typing import Literal
import os
import sys
import numpy as np

sys.path.append(os.path.join(os.path.curdir, ".."))

from search.random_search import RandomSearch
from scripts.run_experiment import prepare_dataset
from models.cnn import CNNModel, TrainingConfig
from models.factory import get_model_by_name
from models.decision_tree import DecisionTreeModel
from models.knn import KNNModel

## Quick Setup - Load Minimal Data

In [2]:
# Load CIFAR-10 dataset

cifar10_data = prepare_dataset()

# Smaller samples for demo (100 train, 50 validation)
SAMPLE_SIZE = 100
VAL_SAMPLE_SIZE = 50

# Sample from the prepared data
np.random.seed(42)
train_indices = np.random.choice(len(cifar10_data['train_images']), SAMPLE_SIZE, replace=False)
val_indices = np.random.choice(len(cifar10_data['val_images']), VAL_SAMPLE_SIZE, replace=False)

# CNN uses raw images (List[np.ndarray])
X_train: list[np.ndarray] = [cifar10_data['train_images'][i] for i in train_indices]
X_test: list[np.ndarray] = [cifar10_data['val_images'][i] for i in val_indices]

# sklearn uses flattened arrays
X_train_flat: np.ndarray = cifar10_data['train_flat'][train_indices]
X_test_flat: np.ndarray = cifar10_data['val_flat'][val_indices]

# Labels are the same for both
y_train = cifar10_data['train_labels'][train_indices]
y_test = cifar10_data['val_labels'][val_indices]

# Observe Sample Shapes
print(f"CIFAR-10 sample image count: {len(X_train)}")
print(f"CIFAR-10 sample label shape: {y_train.shape}")
print(f"Individual image shape: {X_train[0].shape}")

pixel_min = np.min([np.min(img) for img in X_train])
pixel_max = np.max([np.max(img) for img in X_train])
print(f"CIFAR-10 pixel value ranges: [{pixel_min}, {pixel_max}]")

# Show data types
print(f"Image data type: {X_train[0].dtype}")
print(f"Label data type: {y_train.dtype}")

print("Sample datasets loaded successfully!")
print(f"CIFAR-10 sample: {len(X_train)} train, {len(X_test)} validation")


CIFAR-10 sample image count: 100
CIFAR-10 sample label shape: (100,)
Individual image shape: (32, 32)
CIFAR-10 pixel value ranges: [0.0, 1.0]
Image data type: float32
Label data type: int64
Sample datasets loaded successfully!
CIFAR-10 sample: 100 train, 50 validation


## Load All Model Classes

In [5]:
# Silence warnings to avoid printing full paths
import warnings
warnings.filterwarnings("ignore")

# Import all model classes
from models.decision_tree import DecisionTreeModel
from models.knn import KNNModel
from models.cnn import CNNModel


def create_sample_models():
    """Create instances of all model classes for testing"""
    models = {}

    # Decision Tree
    dt_model = DecisionTreeModel()
    dt_model.create_model()
    models["Decision Tree"] = dt_model
    print(f"Decision Tree model created: {dt_model.estimator}")

    # K-Nearest Neighbors
    knn_model = KNNModel()
    knn_model.create_model()
    models["K-Nearest Neighbors"] = knn_model
    print(f"KNN model created: {knn_model.estimator}")

    # CNN (Note: May require special handling due to PyTorch)
    try:
        cnn_model = CNNModel()
        cnn_model.create_model()
        models["Convolutional Neural Network"] = cnn_model
        print(f"CNN model created: {cnn_model.network}")
    except Exception as e:
        print(f"[ERROR] CNN model creation failed: {e}")
        print("CNN will be skipped in quick demo")

    return models


# Create model instances
sample_models = create_sample_models()

print("Model classes loaded successfully!")
for name, model in sample_models.items():
    print(f"  {name}: {type(model).__name__}")

print(f"\nTotal models available: {len(sample_models)}")

Decision Tree model created: DecisionTreeClassifier()
KNN model created: KNeighborsClassifier()
CNN model created: Backbone(
  (block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AdaptiveAvgPool2d(output_size=1)
  )
  (fea

## Hyperparameter Search Test

In [3]:
def quick_hyperparameter_test(
    model_keys: list[Literal['dt', 'knn', 'cnn']],
    X_train: list[np.ndarray],
    y_train: np.ndarray,
    X_test: list[np.ndarray],
    y_test: np.ndarray,
    X_train_flat: np.ndarray,
    X_test_flat: np.ndarray,
    dataset_name: str = "Dataset",
    trials: int = 5
):
    """Perform a quick hyperparameter test using RandomSearch"""
    # Map model keys to display names
    model_key_to_name = {
        "dt": "Decision Tree",
        "knn": "K-Nearest Neighbors",
        "cnn": "Convolutional Neural Network",
    }
    print(f"Starting quick hyperparameter test on {dataset_name}")
    print("=" * 60)
    print(f"Using RandomSearch with {trials} trials per model")
    results = {}
    for model_key in model_keys:
        model_name = model_key_to_name.get(model_key, model_key)
        print(f"\nTesting {model_name}...")
        # Get model and parameter space
        model = get_model_by_name(model_key)
        param_space = model.get_param_space()
        # Create evaluation function for this model
        def evaluate_params(params):
            # Create fresh model instance
            model_instance = get_model_by_name(model_key)
            if model_key == "cnn":
                assert isinstance(model_instance, CNNModel)
                X_train_prep = X_train
                X_test_prep = X_test
                y_train_prep, y_test_prep = y_train, y_test
                # Separate CNN-specific params from training config params
                cnn_params = {}
                training_config_params = {}
                for param_name, param_value in params.items():
                    if param_name in ['batch_size', 'learning_rate', 'optimizer', 'weight_decay']:
                        training_config_params[param_name] = param_value
                    else:
                        cnn_params[param_name] = param_value
                # Create model with CNN architecture params
                model_instance.create_model(**cnn_params)
                # Create training config with training params
                config = TrainingConfig(epochs=5, **training_config_params)
                # Train using the correct CNN signature
                model_instance.train(X_train_prep, y_train_prep, X_test_prep, y_test_prep, config=config, verbose=False)
                # Evaluate CNN
                return model_instance.evaluate(X_test_prep, y_test_prep)
            else:
                assert isinstance(model_instance, (DecisionTreeModel, KNNModel))
                # sklearn models
                X_train_prep = X_train_flat
                X_test_prep = X_test_flat
                y_train_prep, y_test_prep = y_train, y_test
                # Create model with params, then train
                model_instance.create_model(**params)
                model_instance.train(X_train_prep, y_train_prep)
                # Evaluate sklearn models
                return model_instance.evaluate(X_test_prep, y_test_prep)
        # Create and run RandomSearch (sequential)
        random_search = RandomSearch(
            param_space=param_space,
            evaluate_fn=evaluate_params,
            metric_key="accuracy",
            seed=42,  # For reproducibility
            n_jobs=1   # Sequential execution
        )
        # Run the search
        search_result = random_search.run(trials=trials, verbose=True)
        # Store results for this model
        results[model_name] = {
            "best_params": search_result.best_params,
            "best_score": search_result.best_metrics.get("accuracy", 0.0),
            "metrics": search_result.best_metrics,
            "trials": search_result.trials,
            "history": search_result.history
        }
        print(f"Best params: {search_result.best_params}")
        print(f"Best score: {search_result.best_metrics.get('accuracy', 0.0):.4f}")
    print("\n" + "=" * 60)
    print("Quick Hyperparameter Test Summary:")
    for model_name, result in results.items():
        score = result.get("best_score")
        print(f"{model_name}: Best Score = {score:.4f}")
    return results


In [4]:
# Test on CIFAR-10 only
print("\n" + "=" * 60)
print("Testing all models on CIFAR-10 sample...")

# List of model keys to test
model_keys: list[Literal['dt', 'knn', 'cnn']] = ["dt", "knn", "cnn"]

cifar_results = quick_hyperparameter_test(
    model_keys, X_train, y_train, X_test, y_test, X_train_flat, X_test_flat, "CIFAR-10", trials=5
)




Testing all models on CIFAR-10 sample...
Starting quick hyperparameter test on CIFAR-10
Using RandomSearch with 5 trials per model

Testing Decision Tree...
Running 5 trials...
Optimizing for metric: accuracy
Trial 1/5: {'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 5, 'criterion': 'gini'}
Trial 2/5: {'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 2, 'criterion': 'gini'}
Trial 3/5: {'max_depth': 16, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'gini'}
Trial 4/5: {'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 9, 'criterion': 'gini'}
Trial 5/5: {'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 9, 'criterion': 'entropy'}
  -> New best! accuracy=0.2600
Best params: {'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 5, 'criterion': 'gini'}
Best score: 0.2600

Testing K-Nearest Neighbors...
Running 5 trials...
Optimizing for metric: accuracy
Trial 1/5: {'n_neighbors': 23, 'weights': 'uniform', 'metric': 'minkowsk

Noticing the exceptionally low scores, it could be the reason of the **quick** test. It only uses 100 training samples to quickly verify whether the pipeline works normally. So now, we go ahead with a more detailed flow.

## Quick Demo Summary

This quick sample flow demonstrates:

1. **All Model Classes Loaded**: Successfully imported and instantiated all 3 model classes from the `models/` directory
2. **Interface Compliance**: Verified that all models implement the required `BaseModel` interface
3. **Hyperparameter Testing**: Tested hyperparameter tuning functionality with a smaller set of sample data
4. **Training & Evaluation**: Confirmed that all models can train and evaluate on CIFAR-10 data
5. **Use of Baseline**: Only the Random Solver is used for a quicker demo. 

### Models Tested:
- **Decision Tree Model** (`DecisionTreeModel`)
- **K-Nearest Neighbors Model** (`KNNModel`)
- **Convolutional Neural Network Model** (`CNNModel`)

This quick flow uses small sample sizes (100 training, 50 test samples) and limited hyperparameter combinations (max 3 per model) to ensure fast execution while still validating that the complete machine learning pipeline works correctly for all implemented model classes.

---

**Note**: For full experiments, use the comprehensive workflow sections below with complete datasets and extensive hyperparameter search.